## Check Member End Forces
This notebook reads files describing a structure, and the files output by Frame2D after an
analysis, and checks that the member end forces computed here from the displacements and
member loads agree with those computed by Frame2D.

It does this in the simplest way possible, using quite different logic than Frame2D, resulting
in a higher degree of confidence in the results.  It would have been better had someone else
programmed it, but oh well ...

In [10]:
dir = 'KG82'
#dir = 'l22x6'
#dir = 'l22x6pd'

def filename(basename):
    return dir + '.d/' + basename + '.csv'

def Warn(msg):
    print('!!!!! Warning: {}'.format(msg))

In [11]:
import pandas as pd
import math

In [12]:
class Node(object):
    
    def __init__(self,id,x,y):
        self.id = id
        self.x = x
        self.y = y
        self.deltaX = 0.
        self.deltaY = 0.
        self.thetaZ = 0.

In [13]:
table = pd.read_csv(filename('nodes'))
NODES = {}
for i,n in table.iterrows():
    if n.ID in NODES:
        Warn("Node '{}' is multiply defined.".format(n.ID))
    NODES[n.ID] = Node(n.ID,float(n.X),float(n.Y))

In [17]:
table = pd.read_csv(filename('displacements'))
for i,n in table.iterrows():
    node = NODES[n.ID]
    node.deltaX = float(n.DX)
    node.deltaY = float(n.DY)
    node.thetaZ = float(n.RZ)

In [22]:
#pd.DataFrame([vars(v) for v in NODES.values()]).set_index('id')

In [72]:
class Member(object):
    
    E = 200000.
    
    def __init__(self,id,nodej,nodek):
        self.id = id
        self.nodej = nodej
        self.nodek = nodek
        
        dx = nodek.x - nodej.x
        dy = nodek.y - nodej.y
        self.L = L = math.sqrt(dx*dx + dy*dy)
        self.cosx = dx/L
        self.cosy = dy/L
        
        self.Ix = 0.
        self.A = 0.
        self.loads = []
        self.releases = set()

In [73]:
table = pd.read_csv(filename('members'))
MEMBERS = {}
for i,m in table.iterrows():
    if m.ID in MEMBERS:
        Warn("Member '{}' is multiply defined.".format(m.ID))
    MEMBERS[m.ID] = Member(m.ID,NODES[m.NODEJ],NODES[m.NODEK])

In [74]:
import sst
SST = sst.SST()
table = pd.read_csv(filename('properties'))
defIx = defA = None
for i,row in table.iterrows():
    if not pd.isnull(row.SIZE):
        defIx,defA = SST.section(row.SIZE,'Ix,A')
    memb = MEMBERS[row.ID]
    memb.Ix = float(defIx if pd.isnull(row.Ix) else row.Ix)
    memb.A = float(defA if pd.isnull(row.A) else row.A)
    if not pd.isnull(row.Ix):
        defIx = row.Ix
    if not pd.isnull(row.A):
        defA = row.A

In [75]:
table = pd.read_csv(filename('member_loads'))
for i,row in table.iterrows():
    memb = MEMBERS[row.ID]
    typ = row.TYPE
    w1 = None if pd.isnull(row.W1) else float(row.W1)
    w2 = None if pd.isnull(row.W2) else float(row.W2)
    a = None if pd.isnull(row.A) else float(row.A)
    b = None if pd.isnull(row.B) else float(row.B)
    c = None if pd.isnull(row.C) else float(row.C)
    memb.loads.append((typ,w1,w2,a,b,c))

In [76]:
table = pd.read_csv(filename('releases'))
for i,row in table.iterrows():
    memb = MEMBERS[row.ID]
    memb.releases.add(row.R.upper())

In [77]:
t = pd.DataFrame([vars(v) for v in MEMBERS.values()]).set_index('id')
del t['nodej']
del t['nodek']
del t['loads']
t

,A,Ix,L,cosx,cosy,releases
id,,,,,,
EF,12300,222000000,5500,0,1,set([])
AB,12300,222000000,6500,0,1,set([])
KL,12300,222000000,5500,0,1,set([])
IG,12300,222000000,6500,0,1,set([])
LC,13500,488000000,10000,1,0,"set([MZK, MZJ])"
BC,12300,222000000,5500,0,1,set([])
KB,13500,488000000,10000,1,0,"set([MZK, MZJ])"
EG,13500,488000000,10000,1,0,"set([MZK, MZJ])"
DE,12300,222000000,6500,0,1,set([])


In [78]:
MEFS = pd.read_csv(filename('mefs')).set_index('ID')
MEFS

,FXJ,FYJ,MZJ,FXK,FYK,MZK
ID,,,,,,
AB,994654.015809,-2177.821451,4.189959e+07,-994654.015809,2177.821451,-5.605543e+07
BC,452381.814808,-88371.309081,-2.410726e+08,-452381.814808,88371.309081,-2.449696e+08
DE,1055345.984190,63235.567017,1.843676e+08,-1055345.984190,-63235.567017,2.266636e+08
EF,470118.185192,115284.882504,2.959813e+08,-470118.185192,-115284.882504,3.380856e+08
IG,500000.000000,2308.981361,3.725290e-08,-500000.000000,-2308.981361,1.500838e+07
GH,225000.000000,-2728.796153,-1.500838e+07,-225000.000000,2728.796153,1.490116e-08
JK,500000.000000,1933.273074,0.000000e+00,-500000.000000,-1933.273074,1.256627e+07
KL,225000.000000,-2284.777269,-1.256627e+07,-225000.000000,2284.777269,0.000000e+00
CF,90656.086350,227381.814808,2.449696e+08,-90656.086350,245118.185192,-3.380856e+08


In [79]:
for m in MEMBERS.values():
    dX = m.nodek.deltaX - m.nodej.deltaX
    dY = m.nodek.deltaY - m.nodej.deltaY
    ldX = dX*m.cosx + dY*m.cosy
    t = m.E*m.A*ldX/m.L
    m.FXK = t
    m.FXJ = -t

In [87]:
computed = pd.DataFrame([{k:getattr(m,k) for k in 'id FXJ FXK'.split()} 
                         for m in MEMBERS.values()]).set_index('id')
(computed - MEFS[['FXJ','FXK']])

,FXJ,FXK
AB,4.610047e-08,-4.610047e-08
BC,-2.036570e-06,2.036570e-06
BE,-1.291405e-05,1.291405e-05
CF,-4.485322e-06,4.485322e-06
DE,2.215384e-06,-2.215384e-06
EF,-2.436398e-06,2.436398e-06
EG,1.099819e-06,-1.099819e-06
FH,8.398685e-06,-8.398685e-06
GH,9.819050e-07,-9.819050e-07
IG,-8.000643e-07,8.000643e-07
